In [48]:
import pandas as pd

class AcsData:
    def __init__(self,filepath="data/acs.csv"):
        self.filepath = filepath
        self.df = self.load_data()
        self.rename_columns()

    def create_summary(self):
        races = ['white', 'black', 'native', 'pacific_islander', 'other', 'asian','hispanic']

        summary = self.df.groupby('fips').sum()
        for race in races:
            col_name = race + '_percentage'
            summary[col_name] = summary[race] / summary['total_population']

        summary.reindex(columns=['fips'])
        return summary

    def rename_columns(self):
        fips = self.df['GEO.id2'].astype('str').str[:5]

        hispanic_values = ['HD01_VD12','HD01_VD13','HD01_VD14','HD01_VD15','HD01_VD16','HD01_VD17','HD01_VD18',
                           'HD01_VD19','HD01_VD20','HD01_VD21']

        race_cols = ['HD01_VD01','HD01_VD03','HD01_VD04','HD01_VD05','HD01_VD07','HD01_VD08','HD01_VD06']

        combined_cols = hispanic_values + race_cols
        df = self.df[combined_cols]
        df[combined_cols] = df[combined_cols].astype(int)
        df['fips'] = fips

        df['hispanic'] = df[hispanic_values].sum(axis=1)
        df.drop(hispanic_values,axis=1,inplace=True)
        col_names = {
            'HD01_VD01': 'total_population',
            'HD01_VD03': 'white',
            'HD01_VD04': 'black',
            'HD01_VD05': 'native',
            'HD01_VD07': 'pacific_islander',
            'HD01_VD08': 'other',
            'HD01_VD06': 'asian',
            'fips': 'fips',
            'hispanic':'hispanic'
        }

        df.rename(columns=col_names, inplace=True)
        self.df = df

    def load_data(self):
        community_data = pd.read_csv(self.filepath, encoding="ISO-8859-1")
        data = community_data[1:]
        return data

In [31]:
class Stops:
    def __init__(self,filepath):
        self.df = self.load_dataframe(filepath)

    def load_dataframe(self, filepath):
        df = pd.read_csv(filepath)
        cols_to_drop = ['location_raw', 'county_name']
        df = df.drop(cols_to_drop,axis=1)
        return df



In [32]:
filepath = 'data/stop_data/CT-clean.csv'
stops = Stops(filepath)

/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
df = stops.df
df['driver_race_raw'].unique()

array(['Black', 'White', 'Hispanic', 'Native American', 'Asian'],
      dtype=object)

In [49]:
acs = AcsData('data/acs.csv')

/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: DtypeWarning: Columns (1,3,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/shawnveltman/anaconda3/envs/geo/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [50]:
summary = acs.create_summary()
summary.head()

,total_population,white,black,native,pacific_islander,other,asian,hispanic,white_percentage,black_percentage,native_percentage,pacific_islander_percentage,other_percentage,asian_percentage,hispanic_percentage
fips,,,,,,,,,,,,,,,
10001,171474,108515,40690,1024,53,384,3401,23779,0.632836,0.237295,0.005972,0.000309,0.002239,0.019834,0.138674
10003,551997,326993,130912,997,130,1308,28383,106542,0.592382,0.237161,0.001806,0.000236,0.002370,0.051419,0.193012
10005,211224,158057,25948,840,15,608,2379,39438,0.748291,0.122846,0.003977,0.000071,0.002878,0.011263,0.186712
10010,55049,41663,10113,226,6,123,444,2851,0.756835,0.183709,0.004105,0.000109,0.002234,0.008066,0.051790
10030,199510,165950,18406,1228,0,305,1325,17856,0.831788,0.092256,0.006155,0.000000,0.001529,0.006641,0.089499
